In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import qr
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, classification_report
from scipy.optimize import minimize

## Qual o objetivo de estimar o vetor de parametros β^?
 O objetivo da análise de regressão é estimar o vetor β^, que é uma aproximação dos parâmetros populacionais verdadeiros β. β^ é calculado para que o modelo de regressão ajuste-se melhor aos seus dados observados. A relação entre os dois pode ser representada pela equação do modelo de regressão:

Y = Xβ + E

Y: Variável dependente observada.
X: Matriz de variáveis independentes observadas.
β^: Vetor de coeficientes estimados com base nos dados.
E: Vetor de erros.

#Tratamento da base de dados

In [ ]:
df = pd.read_csv("insurance_treino.csv", sep=';', decimal=',')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,34,female,23.56,0,no,northeast,4992.3764
1,45,female,33.10,0,no,southwest,7345.0840
2,23,male,32.70,3,no,southwest,3591.4800
3,38,female,19.95,2,no,northeast,7133.9025
4,32,female,29.80,2,no,southwest,5152.1340


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1000 non-null   int64  
 1   sex       1000 non-null   object 
 2   bmi       1000 non-null   float64
 3   children  1000 non-null   int64  
 4   smoker    1000 non-null   object 
 5   region    1000 non-null   object 
 6   charges   1000 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 54.8+ KB


In [ ]:
# Convertendo categorias em variáveis numéricas
for col in df.columns:
    # Verifica se o tipo da coluna é object (textual)
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1000 non-null   int64  
 1   sex       1000 non-null   int64  
 2   bmi       1000 non-null   float64
 3   children  1000 non-null   int64  
 4   smoker    1000 non-null   int64  
 5   region    1000 non-null   int64  
 6   charges   1000 non-null   float64
dtypes: float64(2), int64(5)
memory usage: 54.8 KB


##Solução analitica matricial

B=(X′X)−1X′Y

- B é um vetor que contém os coeficientes estimados do modelo de regressão linear.
- X é a matriz de design, que contém as covariáveis (variáveis independentes).
- X' é a matriz transposta de X.
- Y é o vetor da variável dependente.
- (X'X)^(-1) é a inversa da matriz resultante da multiplicação de X' por X.

Essa equação é derivada usando a técnica de mínimos quadrados. O objetivo da regressão linear é encontrar os melhores coeficientes (B) que minimizam a soma dos quadrados das diferenças entre as previsões do modelo (X * B) e os valores reais (Y). O processo de minimização leva à equação acima.

In [ ]:
# Separar os dados de treinamento em matriz X (variáveis independentes) e vetor y (variável dependente)
X = df[["age", "bmi", "children"]].values  # Seleciona as variáveis preditoras
X = np.column_stack([X, np.ones(X.shape[0])])  # Adiciona uma coluna de 1s para representar o intercepto
y = df["charges"].values  # Variável de resposta

# Encontre β^ usando a solução analítica
beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
# X.T = matriz transposta de X.
# X.T @ X: Aqui, está multiplicando a matriz transposta X.T pela matriz original X
# Essa multiplicação é essencial para encontrar β^

#Segunda opção
'''
Y = df_treino['charges']
X = matriz_x(['age', 'sex','children','smoker'], df_treino)
Xlx = X.T @ X
inv_xlx = inv(Xlx)
B = inv_xlx @ X.T @ Y
B
'''
print("Vetor de coeficientes β^:", beta_hat)




Vetor de coeficientes β^: [  229.88443472   318.96053775   675.87312731 -6516.51019257]


In [ ]:
# Terceira opção
# Separe os dados de treinamento em matriz X (variáveis independentes) e vetor y (variável dependente)
X = df.drop("charges", axis=1).values  # Seleciona todas as colunas, exceto "charges"
X = np.column_stack([X, np.ones(X.shape[0])])  # Adiciona uma coluna de 1s para representar o intercepto
y = df["charges"].values  # Variável de resposta

# Encontre β^ usando a solução analítica
beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)

print("Vetor de coeficientes β^:", beta_hat)

Vetor de coeficientes β^: [   252.79693433   -242.43596534    353.61900763    564.6409944
  23886.00650328   -456.84884383 -12375.26385411]


##Decomposição QR

A utilização da Decomposição é destinada a reduzir o custo computacional, principalmente qunado a matriz cresce em dimenções.

A decomposição QR é uma técnica que descompõe uma matriz em duas matrizes, uma matriz ortogonal (Q) e uma matriz triangular superior (R)

Martriz ortogonal - sua tranposta é igual a inversa; o que significa que Q.T @ Q é uma matriz identidade.

Matriz triangular superior -  todos os elementos abaixo da diagonal são zero

In [ ]:
#  3 colunas de X para a decomposição QR
X_subset = df[['age', 'sex', 'smoker']].values

# Realize a decomposição QR da matriz de design X_subset
Q, R = qr(X_subset)

# Agora, você pode usar R para resolver o sistema linear e encontrar β^
beta_hat = np.linalg.solve(R, Q.T @ y)
# Q.T @ y - Multiplica a transposta de Q pela variável de resposta y.
# np.linalg.solve(R, Q.T @ y) - Resolve o sistema linear usando a matriz triangular superior R e o resultado da etapa anterior
print("Vetor de coeficientes β^:", beta_hat)

LinAlgError: ignored

##Gradiente descendente


O Gradiente Descendente é uma técnica de otimização usada para ajustar os parâmetros de um modelo de regressão linear (ou outros modelos) de forma a minimizar a função de custo, que geralmente é a função de mínimos quadrados.

O modelo de regressão linear foi ajustado usando a função LinearRegression da biblioteca scikit-learn, que já implementa um método de otimização para encontrar os coeficientes do modelo. No entanto, você pode aplicar o Gradiente Descendente manualmente se desejar ter mais controle sobre o processo de otimização.

In [ ]:
# Crie um modelo de regressão linear
model = LinearRegression()

# Ajuste o modelo aos dados de treinamento
model.fit(X, y)

# Obtenha o vetor de coeficientes β^
beta_hat = np.append(model.coef_, model.intercept_)

print("Vetor de coeficientes β^:", beta_hat)

LinearRegression()

Vetor de coeficientes β^: [   252.79693433   -242.43596534    353.61900763    564.6409944
  23886.00650328   -456.84884383      0.         -12375.26385411]


LinearRegression do scikit-learn, já realiza a otimização internamente. Se você quiser implementar o Gradiente Descendente manualmente para um modelo de regressão linear, precisaria definir explicitamente a função de custo e implementar o loop iterativo de otimização.

No entanto, para a maioria dos casos, a implementação fornecida pelo scikit-learn é suficiente e eficaz

In [ ]:
# Carregue os dados em um DataFrame de teste
df_test = pd.read_csv('insurance_teste.csv', sep=';', decimal=',')

# Variáveis categóricas (gênero e status de fumante) transformadas em variáveis binárias
df['gender'] = pd.get_dummies(df['sex'], drop_first=True)  # Codificação do gênero
df['smoker'] = pd.get_dummies(df['smoker'], drop_first=True)  # Codificação do status de fumante

# Split dos dados em conjuntos de treinamento e teste
X = df[['age', 'gender', 'children', 'smoker']]  # Variáveis independentes
y = df['charges']  # Variável dependente
y_pred = df_test['charges']

# Ajuste o modelo de regressão linear aos dados de treinamento
model = LinearRegression()
model.fit(X, y)

# Previsões no conjunto de teste
y_pred = model.predict(X)

# Avaliação o modelo
mse = mean_squared_error(y, y_pred)
print(f"Erro Quadrático Médio (MSE): {mse:.2f}")

#Segunda opção
'''
le = LabelEncoder()
for col in colunas_textuais:
    df_test[col] = le.fit_transform(df_test[col])

X_teste = matriz_x(['age', 'sex','children','smoker'], df_test)
Vetor de coeficientes β^ = (copiar e colar do resultado do codigo anterior)

Y_predito = X_teste @ B

erro = df_test['charges'] - Y_pred
print("MSE: ",np.mean(erro**2))
'''

LinearRegression()

Erro Quadrático Médio (MSE): 37875141.04


#Regressão Logistica

In [11]:
fd = pd.read_csv("bank_customer_treino.csv")
fd_test = pd.read_csv("bank_custoter_teste.csv")

In [12]:
# Convertendo categorias em variáveis numéricas
for col in fd.columns:
    # Verifica se o tipo da coluna é object (textual)
    if fd[col].dtype == 'object':
        le = LabelEncoder()
        fd[col] = le.fit_transform(fd[col])

In [13]:
# Convertendo categorias em variáveis numéricas
for col in fd_test.columns:
    # Verifica se o tipo da coluna é object (textual)
    if fd_test[col].dtype == 'object':
        le = LabelEncoder()
        fd_test[col] = le.fit_transform(fd_test[col])

In [14]:
# Selecionar as covariáveis (variáveis independentes)
X_train = fd[['gender', 'age', 'credit_card', 'credit_score']]
X_test = fd_test[['gender', 'age', 'credit_card', 'credit_score']]
# Variável dependente (classe-alvo)
y_train = fd['churn']
y_test = fd_test['churn']
# Inicializar e ajustar o modelo de regressão logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Acurácia: {accuracy:.2f}")
print("Relatório de Classificação:\n", report)


LogisticRegression()

Acurácia: 0.78
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.80      0.96      0.88      3205
           1       0.27      0.05      0.09       795

    accuracy                           0.78      4000
   macro avg       0.54      0.51      0.48      4000
weighted avg       0.70      0.78      0.72      4000



In [16]:
# Defina a função de verossimilhança negativa
def negative_log_likelihood(beta, X, y):
    p = 1 / (1 + np.exp(-X @ beta))
    return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

# Defina a função para calcular o Jacobiano
def jacobian(beta, X, y):
    p = 1 / (1 + np.exp(-X @ beta))
    return X.T @ (p - y)

# Inicialize os coeficientes β
initial_beta = np.zeros(X_train.shape[1])

# Minimize a função de verossimilhança negativa usando o método de Newton-CG e o Jacobiano
result = minimize(negative_log_likelihood, initial_beta, args=(X_train, y_train), method='Newton-CG', jac=jacobian)

# Os coeficientes estimados β^ estão em result.x
beta_hat_newton = result.x

# Imprima os coeficientes estimados
print("Coeficientes estimados β^:", beta_hat_newton)

Coeficientes estimados β^: [-0.00060578  0.02675856 -0.00015732 -0.00369965]


In [ ]:
#Segunda opção
'''
def calcula_p(X,B):
   return np.exp(X @ B) / (1+np.exp(X @ B))

def matriz_x(coluna, dados):
    n_ = dados.shape[0]

    col_1 = np.ones(n_)
    col_var = dados[coluna]

    X_mat = np.c_[col_1, np.array(col_var)]

    return X_mat

def beta_update(X, W, y, p):
    return inv(X.T @ W @ X) @ X.T @ (y-p)

X_mat = matriz_x(coluna=["gender", "age", "credit_card", "estimated_salary"], dados=df)

n_col_X = X_mat.shape[1]
betas = np.zeros(n_col_X)

p = calcula_p(X_mat, betas)

W = np.diag(p)

# Beta inicial
betas = np.zeros(n_col_X)

for i in range(30):
    p = calcula_p(X_mat, betas)
    W = np.diag(p)
    beta_k1 = beta_update(X_mat, W, y, p)
    betas = betas + beta_k1

np.round(betas,3)
'''


In [22]:
import numpy as np

# Defina a função de verossimilhança negativa e seu gradiente
def negative_log_likelihood(beta, X, y):
    p = 1 / (1 + np.exp(-X @ beta))
    return -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

def gradient(beta, X, y):
    p = 1 / (1 + np.exp(-X @ beta))
    return X.T @ (p - y)

# Inicialize os coeficientes β
initial_beta = np.zeros(X_train.shape[1])

# Defina a taxa de aprendizado e o número máximo de iterações
learning_rate = 0.01
max_iterations = 1000

# Execute o Gradiente Descendente
beta_hat_gradient = initial_beta
for iteration in range(max_iterations):
    gradient_beta = gradient(beta_hat_gradient, X_train, y_train)
    beta_hat_gradient -= learning_rate * gradient_beta


# Coeficientes estimados após as iterações
print("Coeficientes estimados finais:", beta_hat_gradient)


Coeficientes estimados finais: gender          -1151.034788
age             66329.671727
credit_card      -144.530053
credit_score   -15063.592167
dtype: float64


In [ ]:
# Se quiser ver todos as interações: adcionar essa parte ao for:
#exibir todos os resultados
    '''if (iteration + 1) % 10 == 0:  # Imprimir a cada 10 iterações ou ajuste conforme necessário
        negative_log_likelihood_value = negative_log_likelihood(beta_hat_gradient, X_train, y_train)
        print(f"Iteração {iteration + 1}: Coeficientes estimados: {beta_hat_gradient}, Valor da Função de Verossimilhança Negativa: {negative_log_likelihood_value}")
'''

#Exercicio 4


###Newton-Raphson para funções -> $X = X - \frac{F(x)}{F'(x)}$

####$x = \sqrt10$ => $x^2 = 10$ => $x^2 - 10 = 0$

####Logo: $F(x) = x^2 - 10$ e $F'(x) = 2x$

####Aplicando na fórmula => $x = x - \frac{(x^2 - 10)}{2x}$


####$x = x - \frac{(x^2 - 10)}{2x}$ => $x = x -\frac{x^2}{2x} + \frac{10}{2x}$
<br/>
$x = x - \frac{x}{2} +\frac{10}{2x}$
<br/>
<br/>
$x = \frac{x}{2} + \frac{10}{2x}$
<br/>
<br/>
Finally: $ x = \frac{1}{2}(x + \frac{10}{x}) $

In [23]:
def x_update(x):
    return 1/2 * (x + 10/x)

x_ = 3 #X inicial

for i in range(20):
    x_ = x_update(x_)

print("Valor estimado da raiz de 10 com 10 casas de precisão: ", round(x_,10))

Valor estimado da raiz de 10 com 10 casas de precisão:  3.1622776602


#Exercicio 5

$f(x)=3x^{1/3}=0 \\
f'(x)=x^{-2/3} \\
x_{n+1}=x_n-\frac{3x_n^{1/3}}{x_n^{-2/3}} \\
x_{n+1}=x_n-3x_n^{1/3+2/3} \\
x_{n+1}=x_n-3x_n \\
x_{n+1}=-2x_n$

In [24]:
def x_update(x):
    # return (3*x**(1/3)) / (x**(-2/3))
    return -2*x

In [25]:
x_ = .1

for i in range(10):
    x_ = x_update(x_)
    print(f'Passo {i+1}: {x_:.2f}')

Passo 1: -0.20
Passo 2: 0.40
Passo 3: -0.80
Passo 4: 1.60
Passo 5: -3.20
Passo 6: 6.40
Passo 7: -12.80
Passo 8: 25.60
Passo 9: -51.20
Passo 10: 102.40


#Exercio 6

$f(x)=x^4-5x^3+9x+3 \\
f'(x)=4x^3-15x^2+9=0$

In [26]:
def x_update(x):
    return (x**4 - 5*x**3 + 9*x + 3) / (4*x**3 - 15*x**2 + 9)

In [28]:
def x_update(x):
    return (x**4 - 5*x**3 + 9*x + 3) / (4*x**3 - 15*x**2 + 9)

x_ = 4

for i in range(10):
    x_ = x_ - x_update(x_)
    print(i+1,x_)

1 5.0
2 4.641791044776119
3 4.5375439592074756
4 4.528973727325634
5 4.528917959646293
6 4.528917957294362
7 4.528917957294362
8 4.528917957294362
9 4.528917957294362
10 4.528917957294362


##Regressão Linear

### Solução Analítica &rarr; $\beta = (X'X)^{-1} X'Y$
### Gradiente &rarr; $\beta = \beta - \alpha 2X'(X\beta-Y)$
### Newton-Raphson &rarr; $\beta = \beta - X'(X\beta-Y)(\beta'X')^{-1}$

##Regressão Logistica

### Solução Analítica &rarr; Não tem
### Gradiente &rarr; $\beta = \beta - \alpha [\frac{-1}{n} X'(y-p)]$
### Newton-Raphson &rarr; $\beta = \beta + (X'WX)^{-1} X'(y-p)$

##Raiz/Minimo de função

### Gradiente &rarr; $X = X - \alpha \cdot F(X) $
### Newton-Raphson &rarr; $X = X - \frac{F(X)}{F'(X)}$